In [1]:
### Cu 003 processing ###


#%% load the packages
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec

import defdap.hrdic as hrdic
import defdap.ebsd as ebsd
import defdap.experiment as experiment

from pathlib import Path

import copy 
import pandas as pd
from datetime import datetime

import os

# get dictools stuff 
import sys
sys.path.append("c:/work/hrdic-tools/")
import dictools

plt.rcParams['svg.fonttype'] = 'none'

%matplotlib qt

Loading ITKPyBase... Loading ITKPyBase... Loading ITKCommon... Loading ITKCommon... Loading ITKStatistics... Loading ITKStatistics... Loading ITKImageFilterBase... Loading ITKImageFilterBase... Loading ITKTransform... Loading ITKTransform... Loading ITKImageFrequency... Loading ITKImageFrequency... Loading ITKIOImageBase... Loading ITKIOBMP... Loading ITKIOBMP... Loading ITKIOBioRad... Loading ITKIOBioRad... Loading ITKIOBruker... Loading ITKIOBruker... Loading ITKIOGDCM... Loading ITKIOGDCM... Loading ITKIOIPL... Loading ITKIOIPL... Loading ITKIOGE... Loading ITKIOGE... Loading ITKIOGIPL... Loading ITKIOGIPL... Loading ITKIOHDF5... Loading ITKIOHDF5... Loading ITKIOJPEG... Loading ITKIOJPEG... Loading ITKIOJPEG2000... Loading ITKIOJPEG2000... Loading ITKIOTIFF... Loading ITKIOTIFF... Loading ITKIOLSM... Loading ITKIOLSM... Loading ITKIOMINC... Loading ITKIOMINC... Loading ITKIOMRC... Loading ITKIOMRC... Loading ITKIOMeta... Loading ITKIOMeta... Loading ITKIONIFTI... Loading ITKIONIFTI

<itkTemplate itk::TileConfiguration>
Options:
  [2,]
  [3,]
  [4,]


In [2]:
gaugeLength = 12.0      # mm
gaugeWidth = 1.5        # mm
gaugeThickness = 1.1035 # mm
rig_data_file = './ofhc_cu_5pc.csv'

imaging_wait_time = 300 # s

gauge_csa = gaugeWidth*gaugeThickness*1e-6 # m^2

# rigdata = dictools.expDat(rig_data_file,gaugeLength,gaugeWidth,gaugeThickness)

In [84]:
exp = experiment.Experiment()

dic_dir = Path('./DIC/')
ebsd_pre_dir = Path('./Pre_EBSD/')
ebsd_post_dir = Path('./Post_EBSD/')

dic_frame = experiment.Frame()
for dic_file in sorted(dic_dir.glob('map_*.txt')):
    hrdic.Map(dic_file, experiment=exp, frame=dic_frame,data_type='openpiv')
    
hfw = 20 # microns
pixelwidth = 2048
pixelsize = 20/2048

for inc, dic_map in exp.iter_over_maps('hrdic'):
    dic_map.set_scale(pixelsize)
    dic_map.set_crop(left=100,right=100,top=100,bottom=100)
    # dic_map.plot_map('max_shear',vmin=0,vmax=0.01,plot_scale_bar=True)
    print(dic_map)
    
ebsd_pre_frame = experiment.Frame()
ebsd.Map(ebsd_pre_dir / 'map.cpr',increment=exp.increments[0], frame=ebsd_pre_frame)

Loaded OpenPIV n/a data (dimensions: 1999 x 1999 pixels, sub-window size: 16 x 16 pixels)
Loaded OpenPIV n/a data (dimensions: 1999 x 1999 pixels, sub-window size: 16 x 16 pixels)
Loaded OpenPIV n/a data (dimensions: 1999 x 1999 pixels, sub-window size: 16 x 16 pixels)
Loaded OpenPIV n/a data (dimensions: 1999 x 1999 pixels, sub-window size: 16 x 16 pixels)
Loaded OpenPIV n/a data (dimensions: 1999 x 1999 pixels, sub-window size: 16 x 16 pixels)
Loaded OpenPIV n/a data (dimensions: 1999 x 1999 pixels, sub-window size: 16 x 16 pixels)
Loaded OpenPIV n/a data (dimensions: 1999 x 1999 pixels, sub-window size: 16 x 16 pixels)
Loaded OpenPIV n/a data (dimensions: 1999 x 1999 pixels, sub-window size: 16 x 16 pixels)
Loaded OpenPIV n/a data (dimensions: 1999 x 1999 pixels, sub-window size: 16 x 16 pixels)
Loaded OpenPIV n/a data (dimensions: 1999 x 1999 pixels, sub-window size: 16 x 16 pixels)
Loaded OpenPIV n/a data (dimensions: 1999 x 1999 pixels, sub-window size: 16 x 16 pixels)
Loaded Ope

In [85]:
dic_map = exp.increments[-1].maps['hrdic']
dic_map.set_homog_point(vmin=0,vmax=0.2)

ebsd_map = exp.increments[0].maps['ebsd']
ebsd_map.set_homog_point()

In [88]:
dic_frame.homog_points = [(1722, 1177),
 (1335, 1313),
 (948, 1745),
 (549, 1661),
 (245, 1680),
 (233, 1300),
 (579, 1158),
 (443, 756),
 (559, 136),
 (83, 359),
 (1438, 495),
 (857, 165)]

[(1722, 1177),
 (1335, 1313),
 (948, 1745),
 (549, 1661),
 (245, 1680),
 (233, 1300),
 (579, 1158),
 (443, 756),
 (559, 136),
 (83, 359),
 (1438, 495),
 (857, 165)]

In [89]:
ebsd_pre_frame.homog_points = [(2541, 1823),
 (2240, 1932),
 (1940, 2266),
 (1631, 2207),
 (1394, 2225),
 (1388, 1933),
 (1659, 1820),
 (1554, 1519),
 (1648, 1054),
 (1279, 1219),
 (2323, 1319),
 (1875, 1080)]

[(2541, 1823),
 (2240, 1932),
 (1940, 2266),
 (1631, 2207),
 (1394, 2225),
 (1388, 1933),
 (1659, 1820),
 (1554, 1519),
 (1648, 1054),
 (1279, 1219),
 (2323, 1319),
 (1875, 1080)]

In [3]:
# try with dataframes
df = pd.read_csv(rig_data_file)

df.drop(columns = ['Strain (%)','Stress (MPa)','Corrected Elongation (µm)','Corrected Strain (%)'],inplace=True)

df['Eng Stress / MPa']  = df['Force (N)']*1e-6/gauge_csa
df['Eng Strain / - ']   = (df['Elongation (µm)']-df['Elongation (µm)'][0])*1e-3 / gaugeLength

df['True Stress / MPa'] = df['Eng Stress / MPa']*(1 + df['Eng Strain / - '])
df['True Strain / - ']  = np.log(1+df['Eng Strain / - '])

In [4]:
# get steps with images from DIC file names 
dic_dir = './DIC'
dic_file_list = os.listdir(dic_dir)
dic_file_list.remove('images')

# get values for start of each step 
im_steps = np.asarray([x[4:6] for x in dic_file_list],dtype=int)
imaging_steps_df = df[df['Step'].isin(im_steps)]

# imaging step starts 
imaging_step_starts_df = imaging_steps_df.groupby(['Step']).first()

#force max
idx = imaging_steps_df.groupby(['Step'])['Force (N)'].transform(max) == imaging_steps_df['Force (N)']
imaging_maxforce_df = imaging_steps_df[idx]

# imaging starts 
imaging_start_times



In [46]:
plt.plot(imaging_steps_df['Elongation (µm)'],'+')
plt.plot(imaging_maxforce_df['Elongation (µm)'],'x')

In [83]:
t1 = imaging_maxforce_df['Elapsed Time (s)'].to_numpy()+imaging_wait_time # <------ this might not work given we only wait on some steps! will need the metadata from the raw images to get this
t2 = np.isclose(df['Elapsed Time (s)'],t1[0],atol=0.5) 
df[t2]

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [42]:
plt.plot(times,'+')
plt.plot(image_start_time,'+')

In [8]:
plt.figure()
plt.plot(imaging_starts_df['Eng Strain / - '],imaging_starts_df['Eng Stress / MPa'],'+')
plt.plot(imaging_maxforce_df['Eng Strain / - '],imaging_maxforce_df['Eng Stress / MPa'],'x')
plt.plot(df['Eng Strain / - '],df['Eng Stress / MPa'],'.')

In [10]:
xvar = 'Elapsed Time (s)'
yvar = 'Eng Stress / MPa'

plt.figure()
plt.plot(imaging_starts_df[xvar],imaging_starts_df[yvar],'+')
plt.plot(imaging_maxforce_df[xvar],imaging_maxforce_df[yvar],'x')
plt.plot(df[xvar],df[yvar],'-')